# Hypotheses Ontology

First we prepare libraries that we will be using.

In [1]:
## Uncomment if you do not have owlrl installed (you should have it installed from the RDFS tutorial)
# import sys
# !{sys.executable} -m pip install rdflib  owlready2 pandas
import pandas as pd
from rdflib.namespace import DC, FOAF, NamespaceManager

from owlready2 import *
from rdflib import Graph, Literal, Namespace, RDF, URIRef, OWL, RDFS
# documentation : https://pythonhosted.org/Owlready2/

* Owlready2 * Warning: optimized Cython parser module 'owlready2_optimized' is not available, defaulting to slower Python implementation


Loading and creating ontologies and namespaces.

In [2]:
hypothesis_uri = URIRef("http://example.org/hypothesis_ontology/")
hypothesis_namespace = Namespace(hypothesis_uri)
hypothesis_ontology = get_ontology(hypothesis_uri)

dct_uri = URIRef("http://purl.org/dc/terms/")
dct_namespace = Namespace(dct_uri)
# File from webpage did not work for me -- I downloaded it.
dct_ontology = get_ontology("./imported_ontologies/dublin_core_terms.rdf").load()

frbr_uri = URIRef("http://purl.org/vocab/frbr/core#")
frbr_namespace = Namespace(frbr_uri)
# File from webpage did not work for me -- I downloaded it.
frbr_ontology = get_ontology("./imported_ontologies/frbr-core-20050810.rdf").load()

owl_uri = URIRef("http://www.w3.org/2002/07/owl#")
owl_namespace = Namespace(owl_uri)

fabio_uri = URIRef("http://purl.org/spar/fabio/")
fabio_namespace = Namespace(fabio_uri)

prov_uri = URIRef("http://www.w3.org/ns/prov#")
prov_namespace = Namespace(prov_uri)
# File from webpage did not work for me -- I downloaded it.
prov_ontology = get_ontology("./imported_ontologies/prov_ontology.rdf").load()

* Owlready2 * WARNING: ObjectProperty http://www.w3.org/ns/prov#wasRevisionOf belongs to more than one entity types: [owl.AnnotationProperty, owl.ObjectProperty, prov_ontology.wasDerivedFrom]; I'm trying to fix it...
* Owlready2 * WARNING: ObjectProperty http://www.w3.org/ns/prov#specializationOf belongs to more than one entity types: [owl.AnnotationProperty, owl.ObjectProperty, prov_ontology.alternateOf, owl.topObjectProperty]; I'm trying to fix it...


# Ontology classes and properties

## 1. Classes

In [3]:
hypothesis_ontology.label = "Hypothesis Generation Ontology"

with hypothesis_ontology:
    class Hypothesis(Thing):
        label = "Hypothesis"
        comment = "Paper's hypothesis."
    #   Will include dct:Title, bibo:doi, bibo:pmid, fabio:hasPubMedId, fabio:hasPubMedCentralId, schema:url, foaf:sha1
    #   Will link with fabio:Abstract by oa:hasSource.
    
    class Paper(Thing):
        label = "Research paper or article"
        comment = "General class for all papers"
        
    class Provenance(Thing):
        label = "Provenance"
        comment = "Provenance of the paper."

## 2. Properties

In [4]:
with hypothesis_ontology:
    class contains(ObjectProperty):
        label = "Resource representing a hypothesis's entity set."
        comment = "Links hypothesis with its entity set."
        domain = [Hypothesis]
        
    class isHypothesisOf(ObjectProperty, FunctionalProperty):
        domain = [Hypothesis,]
        range = [Paper,]
        label = "Resource representing a hypothesis' paper."
        comment = "Links hypothesis with paper it comes from."
        
    class isProvenanceOf(ObjectProperty, FunctionalProperty, Provenance >> Paper):
        label = "Resource representing a provenance's paper."
        comment = "Links provenance with the paper it is provenance of."
        
    class hasHypothesis(ObjectProperty, list(frbr_ontology.properties())[45], FunctionalProperty):
        label = "Resource representing a document's hypothesis entity."
        comment = "Links paper with its hypothesis entity."
        domain = [Paper]
        range = [Hypothesis,]
        inverse_property = isHypothesisOf
        
    class hasProvenance(ObjectProperty, FunctionalProperty, Paper >> Provenance):
        label = "Resource representing a paper's provenance."
        comment = "Links paper with its provenance."
        inverse_property = isProvenanceOf

## 2.5. Restrictions

In [5]:
with hypothesis_ontology:
    Hypothesis.equivalent_to = [contains.min(1, ),]

## 3. Graph building

In [6]:
hypothesis_ontology.save(file = "hypothesis_ontology.nt", format = "ntriples") # also supported ntriples
g = Graph()
g.bind("frbr", frbr_namespace)
g.bind("owl", owl_namespace)
g.bind("hyp", hypothesis_namespace)
g.bind("fabio", fabio_namespace)
g.parse("hypothesis_ontology.nt", format="nt")

<Graph identifier=N4cc7cce84789425b8ac6ecdf53a7f7e8 (<class 'rdflib.graph.Graph'>)>

## 4. Additional things added to the graph.

## 5. Saving the graph

In [8]:
result = g.serialize(format='turtle').decode()
print(result)

@prefix fabio: <http://purl.org/spar/fabio/> .
@prefix frbr: <http://purl.org/vocab/frbr/core#> .
@prefix hyp: <http://example.org/hypothesis_ontology/> .
@prefix owl: <http://www.w3.org/2002/07/owl#> .
@prefix rdf: <http://www.w3.org/1999/02/22-rdf-syntax-ns#> .
@prefix rdfs: <http://www.w3.org/2000/01/rdf-schema#> .
@prefix xml: <http://www.w3.org/XML/1998/namespace> .
@prefix xsd: <http://www.w3.org/2001/XMLSchema#> .

<http://example.org/hypothesis_ontology> a owl:Ontology .

hyp:contains a owl:ObjectProperty ;
    rdfs:label "Resource representing a hypothesis's entity set."^^xsd:string ;
    rdfs:comment "Links hypothesis with its entity set."^^xsd:string ;
    rdfs:domain hyp:Hypothesis .

hyp:hasHypothesis a owl:FunctionalProperty,
        owl:ObjectProperty ;
    rdfs:label "Resource representing a document's hypothesis entity."^^xsd:string ;
    rdfs:comment "Links paper with its hypothesis entity."^^xsd:string ;
    rdfs:domain hyp:Paper ;
    rdfs:range hyp:Hypothesis ;
   

In [9]:
hypothesis_owl_file = open("hypothesis_owl.ttl", "w+")
hypothesis_owl_file.write(result)
hypothesis_owl_file.close()